In [2]:
from langchain_community.retrievers import PineconeHybridSearchRetriever


In [24]:
import os 
from pinecone import Pinecone , ServerlessSpec
index_name = "hybrid-search-langchain"
import dotenv
dotenv.load_dotenv()

True

In [4]:
pc = Pinecone(api_key=os.getenv("pinecone_api_key"))

In [5]:
if index_name not in pc.list_indexes().names():
    pc.create_index(name=index_name,dimension=384 , metric='dotproduct' , spec=ServerlessSpec(cloud="aws", region="us-east-1"))

In [6]:
index = pc.Index(index_name)
index

In [ ]:

from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings = GoogleGenerativeAIEmbeddings(model = "models/embedding-001")